# 부록 10.4.4: Claude와 비전을 사용할 때의 모범 사례

비전은 Claude와 상호 작용할 수 있는 새로운 방식을 제공합니다. 이미지에서 최고의 성능을 내기 위한 몇 가지 팁을 모았습니다. 그 전에 먼저 노트북을 실행하는 데 필요한 코드를 설정해 보겠습니다.

In [ ]:
pip install -qUr requirements.txt

In [ ]:
import boto3
from IPython.display import Image
from botocore.exceptions import ClientError

session = boto3.Session()
region = session.region_name

modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
#modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

print(f'Using modelId: {modelId}')
print('Using region: ', region)

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

In [ ]:
def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as f:
        image_file = f.read()

    return image_file

## 멀티모달에 전통적인 기술 적용

역할 할당과 같은 전통적인 프롬프트 엔지니어링 기술을 사용하여 hallucination 문제를 해결할 수 있습니다. 이에 대한 예시를 살펴보겠습니다:


예를 들어 Claude에게 이 이미지에 있는 개의 수를 세도록 하고 싶다고 가정해 봅시다.

In [ ]:
Image(filename='./images/best_practices/nine_dogs.jpg') 

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'jpeg',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/nine_dogs.jpg")}
                },
            },
            {"text": "How many dogs are in this picture?"},
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}
response = bedrock_client.converse(**converse_api_params)

# Extract the generated text content from the response
print(response['output']['message']['content'][0]['text'])

개는 9마리밖에 없지만 Claude는 10마리라고 생각합니다! 프롬프트 엔지니어링을 조금 적용하고 다시 시도해 봅시다.

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'jpeg',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/nine_dogs.jpg")}
                },
            },
            {"text": "You have perfect vision and pay great attention to detail which makes you an expert at counting objects in images. How many dogs are in this picture? Before providing the answer in <answer> tags, think step by step in <thinking> tags and analyze every part of the image."},
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}
response = bedrock_client.converse(**converse_api_params)

print(response['output']['message']['content'][0]['text'])

훌륭합니다! 프롬프트에 프롬프트 엔지니어링을 적용한 후에는 Claude가 개가 9마리라고 정확히 셉니다.

## 시각적 프롬프팅

이미지 입력을 사용하면 이제 이미지 자체에 프롬프트를 제공할 수 있습니다. 몇 가지 예를 살펴보겠습니다.

이 이미지에서 우리는 텍스트를 작성하고 화살표를 그렸습니다. 텍스트 프롬프트 없이 이것을 Claude에 전달해 보겠습니다.

In [ ]:
Image(filename='./images/best_practices/circle.png') 

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'png',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/circle.png")}
                },
            }
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
    "inferenceConfig": {"maxTokens": 2048},
}
response = bedrock_client.converse(**converse_api_params)

print(response['output']['message']['content'][0]['text'])

보시다시피 우리가 질문을 주지 않았기 때문에 Claude는 이미지를 설명하려고 했습니다. 이미지에 질문을 추가하고 다시 전달해 봅시다.

In [ ]:
Image(filename='./images/best_practices/labeled_circle.png') 

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'png',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/labeled_circle.png")}
                },
            }
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
    "inferenceConfig": {"maxTokens": 2048},
}
response = bedrock_client.converse(**converse_api_params)

print(response['output']['message']['content'][0]['text'])

우리는 또한 이미지의 특정 부분을 강조하고 그것에 대한 질문을 할 수 있습니다.

이 두 숫자의 차이는 무엇입니까?

In [ ]:
Image(filename='./images/best_practices/table.png') 

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'png',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/table.png")}
                },
            },
            {"text": "What’s the difference between these two numbers?"},
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}
response = bedrock_client.converse(**converse_api_params)

# Extract the generated text content from the response
print(response['output']['message']['content'][0]['text'])

## 퓨샷 예시

시각적 작업에서도 프롬프트에 예시를 추가하면 정확도가 향상됩니다. Claude에게 속도계 사진을 읽게 해 봅시다.

In [ ]:
Image(filename='./images/best_practices/140.png') 

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'png',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/140.png")}
                },
            },
            {"text": "What speed am I going?"},
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}
response = bedrock_client.converse(**converse_api_params)

# Extract the generated text content from the response
print(response['output']['message']['content'][0]['text'])

Claude의 답변이 잘못된 것 같습니다. 우리가 시속 140km가 아니라 140마일로 가고 있다고 생각합니다! 다시 시도해 보겠지만 이번에는 프롬프트에 예시를 추가해 보겠습니다.

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'png',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/70.png")}
                },
            },
            {"text": "What speed am I going?"},
        ]
    },
    {
        "role": 'assistant',
        "content": [
            {"text": "You are going 70 miles per hour."}
        ]
    },
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'png',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/100.png")}
                },
            },
            {"text": "What speed am I going?"},
        ]
    },
    {
        "role": 'assistant',
        "content": [
            {"text": "You are going 100 miles per hour."}
        ]
    },
    {
        "role": 'user',
        "content": [
            {"image": {
                "format": 'png',
                "source": {"bytes": get_base64_encoded_image("./images/best_practices/140.png")}
                },
            },
            {"text": "What speed am I going?"},
        ]
    },
]


converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}
response = bedrock_client.converse(**converse_api_params)

# Extract the generated text content from the response
print(response['output']['message']['content'][0]['text'])

완벽합니다! 그 예시들 덕분에 Claude가 속도계의 속도를 읽는 방법을 배웠습니다. 그러나 이미지에 대한 퓨샷 프롬프팅이 항상 작동하는 것은 아니지만 사용 사례에서 시도해 볼 만합니다.

## 다중 이미지 입력
Claude는 프롬프트 내에서 한 번에 여러 이미지를 수용하고 추론할 수도 있습니다! 예를 들어, 매우 큰 이미지(예: 긴 영수증 이미지)가 있다고 가정해 봅시다! 우리는 그 이미지를 여러 조각으로 나누고 각 조각을 Claude에 공급할 수 있습니다.

In [ ]:
Image(filename='./images/best_practices/receipt1.png') 

In [ ]:
Image(filename='./images/best_practices/receipt2.png') 

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {"format": 'png',"source": {"bytes": get_base64_encoded_image("./images/best_practices/receipt1.png")}},},
            {"image": {"format": 'png',"source": {"bytes": get_base64_encoded_image("./images/best_practices/receipt2.png")}},},
            {"text": "Output the name of the restaurant and the total."},
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}
response = bedrock_client.converse(**converse_api_params)

# Extract the generated text content from the response
print(response['output']['message']['content'][0]['text'])

## 예시에서 객체 식별

이미지 입력을 사용하면 프롬프트에 다른 이미지를 전달하고 Claude가 그 정보를 사용하여 질문에 답할 수 있습니다. 이에 대한 예시를 살펴보겠습니다.

예를 들어 이미지에서 바지 종류를 식별하려고 한다고 가정해 봅시다. 프롬프트에 다양한 바지 종류의 예시를 Claude에게 제공할 수 있습니다.

In [ ]:
Image(filename='./images/best_practices/officer_example.png') 

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"image": {"format": 'png',"source": {"bytes": get_base64_encoded_image("./images/best_practices/wrinkle.png")}},},
            {"image": {"format": 'png',"source": {"bytes": get_base64_encoded_image("./images/best_practices/officer.png")}},},
            {"image": {"format": 'png',"source": {"bytes": get_base64_encoded_image("./images/best_practices/chinos.png")}},},
            {"image": {"format": 'png',"source": {"bytes": get_base64_encoded_image("./images/best_practices/officer_example.png")}},},
            {"text": "These pants are (in order) WRINKLE-RESISTANT DRESS PANT, ITALIAN MELTON OFFICER PANT, SLIM RAPID MOVEMENT CHINO. What pant is shown in the last image?"},
        ]
    }
]

converse_api_params = {
    "modelId": modelId,
    "messages": messages,
}
response = bedrock_client.converse(**converse_api_params)

# Extract the generated text content from the response
print(response['output']['message']['content'][0]['text'])